In [29]:
import pandas as pd

In [30]:
mapa = pd.read_excel("/content/mapa parque.xlsx")
lead = pd.read_excel("/content/leads asa sul e asa norte.xlsx")

In [32]:
# Padronizar o nome das colunas dos dois dataframes e criar a coluna cliente

lead.rename(columns={
    'CNPJ': 'cnpj',
    'UF': 'uf',
    'Cidade': 'municipio',
    'CEP': 'cep',
    'Tipo': 'tipo endereco',
    'Bairro': 'bairro',
    'CNAE Principal': 'cnae',
    'Matriz/Filial - Ente Federativo': 'matriz',
    'Natureza Jurídica': 'natureza_juridica',
    'Data Início Atv.': 'data_abertura',
    'Data Situação Cad.': 'data_sit_cad',
    'Porte Empresa': 'porte',
    'Capital Social da Empresa': 'capital_social',
    'Regime Tributário': 'regime_tributario',
    'Faturamento Estimado': 'faturamento',
    'Quadro de Funcionários': 'quadro_funcionarios',

}, inplace=True)

lead['cliente'] = 0
lead = lead[mapa.columns]

In [65]:
def remover_caracteres_especiais(x):
  return x.replace('.', '').replace('-', '').replace('/', '')

def matriz_ou_filial(x):
  if 'matriz' in x.lower():
    return 'MATRIZ'

  if 'filial' in x.lower():
    return 'FILIAL'

  return x

In [92]:
na_values = []

In [93]:
for column_lead, column_parque in zip(lead.columns, mapa.columns):
  lead_na_values = lead[column_lead].isna().value_counts().to_dict().get(True, 0)
  mapa_na_values = mapa[column_parque].isna().value_counts().to_dict().get(True, 0)

  na_values_dict = {f'lead_{column_lead}': lead_na_values, f'mapa_{column_parque}': mapa_na_values}
  na_values.append(na_values_dict)

In [95]:
na_values

[{'lead_cnpj': 0, 'mapa_cnpj': 0},
 {'lead_uf': 0, 'mapa_uf': 1},
 {'lead_municipio': 0, 'mapa_municipio': 1},
 {'lead_cep': 0, 'mapa_cep': 1},
 {'lead_tipo endereco': 509, 'mapa_tipo endereco': 450},
 {'lead_bairro': 0, 'mapa_bairro': 2},
 {'lead_cnae': 0, 'mapa_cnae': 1},
 {'lead_matriz': 0, 'mapa_matriz': 1},
 {'lead_natureza_juridica': 96, 'mapa_natureza_juridica': 1},
 {'lead_data_abertura': 0, 'mapa_data_abertura': 1},
 {'lead_data_sit_cad': 0, 'mapa_data_sit_cad': 1},
 {'lead_porte': 0, 'mapa_porte': 1},
 {'lead_capital_social': 0, 'mapa_capital_social': 1},
 {'lead_regime_tributario': 0, 'mapa_regime_tributario': 1},
 {'lead_faturamento': 0, 'mapa_faturamento': 1},
 {'lead_quadro_funcionarios': 0, 'mapa_quadro_funcionarios': 1},
 {'lead_cliente': 0, 'mapa_cliente': 0}]

In [96]:
lead.dropna(inplace=True)
mapa.dropna(inplace=True)

In [103]:
lead['cep'] = lead['cep'].apply(lambda x: remover_caracteres_especiais(x))
mapa['cep'] = mapa['cep'].astype(int)

In [76]:
lead['cnpj'] = lead['cnpj'].apply(lambda x: remover_caracteres_especiais(x))
mapa['cnpj'] = mapa['cnpj'].astype(str).str.zfill(14)

In [56]:
lead['matriz'] = lead['matriz'].apply(lambda x: matriz_ou_filial(x))
mapa['matriz'] = mapa['matriz'].apply(lambda x: {1: 'MATRIZ', 2: 'FILIAL'}.get(x))

In [48]:
mapa['porte'] = mapa['porte'].apply(lambda x: {1: 'MICROEMPRESA', 3: 'PEQUENO PORTE', 5: 'MÉDIO/GRANDE PORTE'}.get(x))
lead['porte'] = lead['porte'].apply(lambda x: {'MICRO EMPRESA': 'MICROEMPRESA', 'MICRO EMPREENDEDOR INDIVIDUAL (MEI)': 'MICROEMPRESA', 'EMPRESA DE PEQUENO PORTE': 'PEQUENO PORTE', 'MÉDIO/GRANDE PORTE': 'MÉDIO/GRANDE PORTE'}.get(x))

In [107]:
# Remover todas as linhas de lead cujo cpnj esteja presente em mapa
lead = lead[~lead['cnpj'].isin(mapa['cnpj'])]

In [109]:
# Concatenar as duas bases
base = pd.concat([mapa, lead])

In [113]:
base.to_excel("base completa regressão.xlsx")